# Model Context Protocol (MCP) Tools

In this notebook, we'll explore the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) in depth and learn how to leverage it effectively with the Strands Agent Framework. MCP is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). It enables communication between the system and locally running MCP servers that provide additional tools and resources to extend the capabilities of LLMs.


## Understanding MCP Architecture

The MCP architecture consists of several key components:

1. **MCP Hosts**: are programs like your Strands agents that want to access data 
2. **MCP Client**: The component that sends requests to MCP servers. In our case, this is integrated into the Strands Agent Framework.
3. **MCP Server**: A service that provides tools and context to LLMs through a standardized API.
4. **MCP Tools**: Specific functionalities exposed by the MCP server that the LLM can use.
5. **Context**: The information provided to the LLM to help it understand how to use the tools.


![mcp](image/mcp.png)

## Let's create a simple MCP server.

In this section, we will do a walkthrough of selected code blocks, that have been used to build [mcp_calculator.py](mcp_calulator.py) to create a simple MCP server that provides limited calculator functionality.

![calculator](image/calculator.png)


### ✅ Create the MCP server 


```python
from mcp.server import FastMCP

mcp = FastMCP("Calculator Server")

@mcp.tool(description="Add two numbers together")
def add(x: int, y: int) -> int:
    """Add two numbers and return the result."""
    return x + y

# Define a subtraction tool
@mcp.tool(description="Subtract one number from another")
def subtract(x: int, y: int) -> int:
        """Subtract y from x and return the result.

        Args:
            x: Number to subtract from
            y: Number to subtract

        Returns:
            The difference (x - y)
        """
    return x - y

# Define a multiplication tool
@mcp.tool(description="Multiply two numbers together")
def multiply(x: int, y: int) -> int:
        """Multiply two numbers and return the result.

        Args:
            x: First number
            y: Second number

        Returns:
            The product of x and y
        """
    return x * y

# Define a division tool
@mcp.tool(description="Divide one number by another")
def divide(x: float, y: float) -> float:
        """Divide x by y and return the result.

        Args:
            x: Numerator
            y: Denominator (must not be zero)

        Returns:
            The quotient (x / y)

        Raises:
            ValueError: If y is zero
        """
    if y == 0:
        raise ValueError("Cannot divide by zero")
    return x / y
```

MCP uses JSON-RPC to encode messages. JSON-RPC messages MUST be UTF-8 encoded

The protocol currently defines two standard [transport](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) mechanisms for client-server communication:

- [Standard Input/Output (stdio)](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports#stdio): enables communication through standard input and output streams. It is particulary useful for local integrations and command-line tools
- [Streamable HTTP](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports#streamable-http) : this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.

> Learn more in [Use Model Context Protocol (MCP) as tools with Strands Agent](https://catalog.us-east-1.prod.workshops.aws/workshops/33f099a6-45a2-47d7-9e3c-a23a6568821e/en-US/01-fundamentals/14-integration-mcp-tools)

[mcp_calculator.py](mcp_calulator.py) uses Streamable HTTP

```python
mcp.run(transport="streamable-http")
```

### ✅ Connecting the server to the Strands Agent

The following code shows how to create a Strands agent, and connect it to a MCP server:

```python
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from strands.models.anthropic import AnthropicModel

model = AnthropicModel(
    client_args={
        "api_key": "<KEY>", #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

    # Create a system prompt that explains the calculator capabilities
    system_prompt = """
    You are a helpful calculator assistant that can perform basic arithmetic operations.
    You have access to the following calculator tools:
    - add: Add two numbers together
    - subtract: Subtract one number from another
    - multiply: Multiply two numbers together
    - divide: Divide one number by another
    
    When asked to perform calculations, use the appropriate tool rather than calculating the result yourself.
    Explain the calculation and show the result clearly.
    """

def create_streamable_http_transport():
   return streamablehttp_client("http://localhost:8000/mcp/")

streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

# Use the MCP server in a context manager
with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()
    
    # Create an agent with the MCP tools
    agent = Agent(model=model,system_prompt=system_prompt,tools=tools)

```




### ✅ Summary

At this point, the agent has successfully connected to the MCP server and retrieved the calculator tools. These MCP tools have been converted into standard AgentTools that the agent can use just like any other tools provided to it. The agent now has full access to the calculator functionality without needing to know the implementation details of the MCP server.

### ✅ Test the calculator agent

Go to the terminal and run the app

```

python3 mcp_calulator.py

```

![calculator](image/calculator.gif)




## Custom tools and strands agents tools
### Lets create a Weather Agent

Strands agents offer an [optional tools package](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/tools_overview/#adding-tools-to-agents). You can also create [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) similar to MCP implementation, but without the need to run a dedicated server.

To create the weather agent, we use the [http_request tool](https://github.com/strands-agents/tools/blob/main/src/strands_tools/http_request.py). This tool connects Strands agents with external web services and APIs, bridging conversational AI with data sources. The tool supports multiple HTTP methods (GET, POST, PUT, DELETE), handles URL encoding and response parsing, and returns structured data from web sources.

![weather_agent](image/weather_agent.png)

In [ ]:
from strands import Agent
from strands_tools import http_request
from strands.models.anthropic import AnthropicModel

# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""

model = AnthropicModel(
    client_args={
        "api_key": "<KEY>", #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

# Create an agent with HTTP capabilities
weather_agent = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT,
    tools=[http_request],  # Explicitly enable http_request tool
)

### ✅ Test the Weather agent

In [ ]:
response = weather_agent("What's the weather like in Seattle?")


In [ ]:
print(response.message['content'][0]['text'])

### ✅  Weather agent with a custom tool

![weather_agent_tool](image/weather_agent_tool.png)

In [ ]:
from strands import Agent
from strands_tools import http_request
from strands.models.anthropic import AnthropicModel
from strands.tools import tool
from typing import Dict, Any

# Define a function to convert Fahrenheit to Celsius
@tool
def fahrenheit_to_celsius(params: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert temperature from Fahrenheit to Celsius.

    Args:
        params: A dictionary containing:
            - fahrenheit: The temperature in Fahrenheit to convert

    Returns:
        A dictionary containing:
            - celsius: The temperature in Celsius
            - original_fahrenheit: The original Fahrenheit value
    """
    try:
        fahrenheit = float(params.get("fahrenheit"))
        celsius = (fahrenheit - 32) * 5/9
        return {
            "celsius": round(celsius, 2),
            "original_fahrenheit": fahrenheit
        }
    except (TypeError, ValueError):
        return {
            "error": "Invalid input. Please provide a valid number for fahrenheit."
        }


# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT_C_TOOL = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States
4. Use fahrenheit_to_celsius to convert temperatures between Fahrenheit and Celsius

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""

model = AnthropicModel(
    client_args={
        "api_key": "<KEY>", #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

# Create an agent with HTTP capabilities
weather_agent_c_tool = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT_C_TOOL,
    tools=[http_request,fahrenheit_to_celsius],  # Explicitly enable http_request tool
)

In [ ]:
response_c_tool = weather_agent_c_tool("What is the weather like in Seattle? Please give me the answer in degrees Celsius")


In [ ]:
print(response_c_tool.message['content'][0]['text'])